In [2]:
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import sys
import csv


In [3]:
from keras.models import load_model
model = load_model('gen_age_final.h5')


# For Image Input

In [19]:
def predict_age_gender_image(image_path, model):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    if len(faces) == 0:
        print("No faces detected.")
        return []

    results = []

    for (x, y, w, h) in faces[:5]: 
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        face = face.reshape((1, 48, 48, 1))
        face = face / 255.0  
        
        age, gender = model.predict(face)
    
        gender_str = "Male" if np.argmax(gender) == 0 else "Female"
        results.append((gender_str, int(age[0][0])))
    for i, (gender, age) in enumerate(results, start=0):
        print(f"For ({image_path}): Gender - {gender}, Age - {age}")

    return results


## Assuming you have all images to be tested in a folder named "test_images"

In [20]:
import os

folder_path = 'test_images'
test_images = []

for file_name in os.listdir(folder_path):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        file_path = os.path.join(folder_path, file_name)
        test_images.append(file_path)

for img_path in test_images:
    
    predict_age_gender_image(img_path, model)


1/1 [==============================] - 0s 35ms/step
For (test_images\b1.jpeg): Gender - Male, Age - 30
1/1 [==============================] - 0s 35ms/step
For (test_images\deepika_test_image.jpeg): Gender - Female, Age - 24
1/1 [==============================] - 0s 43ms/step
For (test_images\download.jpeg): Gender - Female, Age - 23
1/1 [==============================] - 0s 40ms/step
For (test_images\images (1).jpeg): Gender - Male, Age - 32
1/1 [==============================] - 0s 52ms/step
For (test_images\images.jpeg): Gender - Female, Age - 22


# For Video Stream

In [4]:
def detect_and_predict_age_gender_video(frame, face_cascade, model):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    results = []

    for idx, (x, y, w, h) in enumerate(faces[:5], start=1):  # Limiting to 5 faces per frame
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        face = face.reshape((1, 48, 48, 1))
        face = face / 255.0

        age, gender = model.predict([face, face])
        gender_str = "Male" if np.argmax(gender) == 0 else "Female"
        results.append((idx, (x, y), (x + w, y + h), gender_str, int(age[0][0])))

    return results


In [112]:
def process_video_stream(model):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)  # Use 0 for the default camera
    
    frame_data = []
    frame_number = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        
        predictions = detect_and_predict_age_gender_video(frame, face_cascade, model)

        frame_info = {'Frame Number': frame_number}
        for idx, (x1, y1), (x2, y2), gender, age in predictions:
            face_info = {
                f'Face {idx} Bottom Left coordinates': (x1, y1),
                f'Face {idx} Top right coordinates': (x2, y2),
                f'Face {idx} Gender': gender,
                f'Face {idx} Age': age
            }
            frame_info.update(face_info)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{gender}, {age}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            
        frame_data.append(frame_info)
        cv2.imshow("Age and Gender Detection", frame)
        frame_number += 1

        if cv2.waitKey(50) & 0xFF == ord('q'):  # Adjust the delay to control FPS
            break

    cap.release()
    cv2.destroyAllWindows()

    # Writing data to CSV file
    with open('output_raw.csv', 'w', newline='') as csvfile:
        fieldnames = ['Frame Number']
        for idx in range(1, 6):
            fieldnames.extend([f'Face {idx} Bottom Left coordinates', f'Face {idx} Top right coordinates', f'Face {idx} Gender', f'Face {idx} Age'])

        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for frame_info in frame_data:
            writer.writerow(frame_info)


In [113]:
process_video_stream(model)


1/1 [==============================] - 0s 36ms/step


## Ouput CSV File in required format saved to "output.csv"

In [114]:
import csv

def interchange_rows_columns(input_file, output_file):
    with open(input_file, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)

    transposed_data = list(map(list, zip(*data)))

    for row in transposed_data:
        for i, value in enumerate(row):
            if value == 'Male':
                row[i] = 1
            elif value == 'Female':
                row[i] = 0

    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for row in transposed_data:
            writer.writerow(row)

input_file = 'output_raw.csv'
output_file = 'output.csv' # Output file in asked format with 1 for Male and 0 for Female
interchange_rows_columns(input_file, output_file)


## Age MSE and Gender Accuracy Testing of video stream data

In [115]:
import pandas as pd
df = pd.read_csv('output_raw.csv')
df.dropna(how='all')
df.to_csv('output_raw.csv', index = False)


## For Age MSE

In [116]:
import pandas as pd
from sklearn.metrics import mean_squared_error


actual_ages_input = input("Provide list of Actual ages of people tested on (space-separated): ")
actual_ages = [int(age.strip()) for age in actual_ages_input.split(" ")]

mse_values = []
for i, actual_age in enumerate(actual_ages, start=1):
    predicted_col = f'Face {i} Age'
    if predicted_col in df.columns:
        predicted_values = df[predicted_col].apply(pd.to_numeric, errors='coerce').dropna()
        if len(predicted_values) > 0:
            mse = mean_squared_error([actual_age] * len(predicted_values), predicted_values)
            mse_values.append(mse)

if mse_values:
    average_mse = sum(mse_values) / len(mse_values)
else:
    average_mse = 0 

for i,mse in enumerate(mse_values):
    print(f"Face {i + 1} MSE: {mse}")

print(f"Average Mean Squared Error (MSE) for Age up to {len(actual_ages)} faces:", average_mse)


Face 1 MSE: 101.25
Average Mean Squared Error (MSE) for Age up to 1 faces: 101.25


## For Gender Accuracy

In [117]:
male_female_counts = {}
for col in df.columns:
    if col.startswith('Face') and col.endswith('Gender'):
        face_number = int(col.split()[1])
        counts = df[col].value_counts()
        male_female_counts[face_number] = {'Male': counts.get('Male', 0), 'Female': counts.get('Female', 0)}


actual_genders_input = input("Provide list of Actual Gender of people tested on (space-separated): like 'M F M'")
actual_genders_coded = actual_genders_input.split(' ')
actual_genders = ['Male' if actual_gender == 'M' else 'Female' for actual_gender in actual_genders_coded]

accuracy = []
for face_number, counts in male_female_counts.items():
    face_number -= 1  
    accuracy.append(counts[actual_genders[face_number]] / sum(counts.values())) 
    if face_number == len(actual_genders) - 1:  
        break
for face_number, acc in enumerate(accuracy):
    print(f"Face {face_number + 1} Gender Accuracy: {acc:.2%}")

average_accuracy = sum(accuracy)/len(accuracy)
print(f"Average Gender accuracy for up to {len(actual_genders)} faces: {average_accuracy:.2%}")


Face 1 Gender Accuracy: 100.00%
Average Gender accuracy for up to 1 faces: 100.00%
